In [16]:
import sys 
import mlflow
import os 
import hydra
import pandas as pd
from omegaconf import OmegaConf, DictConfig, open_dict
import numpy as np
import torch
from pathlib import Path
from source.models import MLP, GraphTransformer
from source.factories import  dataset_factory, model_factory
from source.factories import optimizers_factory, logger_factory, lr_schedulers_factory
from source.training import SupervisedTrainer
from datetime import datetime
from source.utils import set_seed

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
with hydra.initialize(config_path="source/conf", version_base=None):  # version_base ensures compatibility; adjust if needed
    # Compose the config, specifying config_name here
    cfg: DictConfig = hydra.compose(config_name="config_graph_transformer") 

/Users/irinaknyazeva/Yandex.Disk.localized/Projects/BrainIHBProjects/LabNeuroimagingProjects/DLfMRI/DLBenchmarking/.venv/lib/python3.11/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config_graph_transformer': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [36]:
cfg.dataset

{'name': 'abide', 'batch_size': 40, 'test_batch_size': 20, 'val_batch_size': 20, 'train_set': 0.7, 'val_set': 0.1, 'path': '/Users/irinaknyazeva/Yandex.Disk.localized/Projects/BrainIHBProjects/LabNeuroimagingProjects/DLfMRI/datasets/abide.npy', 'stratified': True, 'node_sz': 200, 'node_feature_sz': 200, 'drop_last': True}

In [37]:
batch_size = 16
# Generate random input data
node_feature = torch.randn(batch_size, 200, 200)  # (batch_size, 200, 200)

In [38]:
model = GraphTransformer(cfg)
model.eval()  # Set to evaluation mode


GraphTransformer(
  (attention_list): ModuleList(
    (0-1): 2 x TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=20, out_features=20, bias=True)
      )
      (linear1): Linear(in_features=20, out_features=128, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=128, out_features=20, bias=True)
      (norm1): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (input_projection): Linear(in_features=200, out_features=20, bias=True)
  (dim_reduction): Sequential(
    (0): Linear(in_features=20, out_features=8, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (fc): Sequential(
    (0): Linear(in_features=1600, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): 

In [45]:
att_res = model.input_projection(node_feature)

In [40]:
model.attention_list[0]

TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=20, out_features=20, bias=True)
  )
  (linear1): Linear(in_features=20, out_features=128, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=128, out_features=20, bias=True)
  (norm1): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

In [ ]:
model.dim_reduction(att_res)

torch.Size([16, 200, 8])

In [49]:
torch.nn.BatchNorm1d(10)

BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [42]:
model(node_feature).shape

torch.Size([16, 2])

In [5]:
#set_seed(cfg.seed)
dataloaders = dataset_factory(cfg)
batch = next(iter(dataloaders['train']))
batch[2][:4], batch[0][0,:2,:2]

(tensor([0, 1, 1, 1]),
 tensor([[-0.0604,  0.0552],
         [-0.4039, -0.6505]]))

(tensor([1, 0, 1, 1]),
 tensor([[0.9210, 0.6161],
         [1.2456, 1.4961]]))

tensor([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1])

In [5]:
res_ = cfg.get("experiment_name", None)
res_

'MLPexperiment'

In [10]:
exp = mlflow.get_experiment_by_name("MLPexperiment")
exp._experiment_id

'941851421475770388'

In [11]:
!mlflow experiments delete -x 941851421475770388

Experiment with ID 941851421475770388 has been deleted.


## Testing logger, dataloader, model, optimizers

In [31]:
logger = logger_factory(cfg)

In [40]:
dataloaders = dataset_factory(cfg)

[2025-08-10 18:57:07,838][<string>][L24][INFO] Using stratified dataloader for 'abide' dataset.


In [33]:
dataloaders

In [22]:
model = model_factory(cfg)

[2025-08-09 09:42:50,859][<string>][L22][INFO] Will be used model class[MLP]


In [23]:
model

MLP(
  (network): Sequential(
    (0): Linear(in_features=19900, out_features=25, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=25, out_features=12, bias=True)
    (4): LeakyReLU(negative_slope=0.01)
    (5): Dropout(p=0.1, inplace=False)
  )
  (fc): Linear(in_features=12, out_features=2, bias=True)
)

In [25]:
optimizers = optimizers_factory(cfg, model=model)

[2025-08-09 09:44:23,397][<string>][L11][INFO] Creating 1 optimizer(s) based on the configuration.
[2025-08-09 09:44:23,399][factories.py][L85][INFO] Parameters [normal] length [6]


In [28]:

lr_schedulers = lr_schedulers_factory(cfg=cfg)

[2025-08-09 09:45:51,002][<string>][L15][INFO] Creating schedulers for 1 optimizer(s)...


In [41]:
dataloaders

In [5]:
with open_dict(cfg):
    cfg.dataset.node_sz = 200 
    

In [47]:
from source.training import SupervisedTrainer

In [42]:
batch = next(iter(dataloaders[0]))

In [48]:
trainer = SupervisedTrainer(cfg, model, optimizers, lr_schedulers, dataloaders,logger)

In [58]:
trainer._preprocess_batch(batch)

[2025-08-11 16:32:02,361][<string>][L12][WARNING] Could not find 'source.prepossess.py'. Skipping preprocessing.


{'time_series': tensor([[[-1.3022e-01, -3.4999e-01, -5.8835e-01,  ...,  2.0667e-01,
            2.5229e-01,  1.9874e-01],
          [ 1.2365e-01, -5.4046e-02, -1.8125e-01,  ...,  2.1649e-01,
            3.0064e-01,  6.9263e-01],
          [-4.4977e-01, -4.6650e-01, -2.7012e-01,  ...,  7.0560e-02,
           -5.2195e-01, -9.5217e-01],
          ...,
          [ 9.2356e-04,  9.2356e-04,  9.2356e-04,  ...,  9.2356e-04,
            9.2356e-04,  9.2356e-04],
          [-1.1930e+00, -1.5629e+00, -1.4659e+00,  ...,  1.3430e+00,
            6.0792e-01,  9.0091e-02],
          [-3.4078e-01, -4.5209e-01, -5.1648e-01,  ..., -3.0792e-01,
           -2.1767e-01,  5.2791e-02]],
 
         [[-5.6952e-01, -9.6260e-01, -9.3498e-01,  ..., -5.3651e-01,
           -8.6565e-02,  5.2540e-01],
          [ 4.0425e-01,  5.1663e-01,  6.5948e-01,  ..., -9.1529e-01,
           -1.0688e+00, -1.0083e+00],
          [-5.3576e-01, -1.3074e-01,  6.0215e-01,  ...,  1.3889e-01,
            6.3007e-01,  5.7869e-01],
    

In [7]:
batch_size = 4  # A typical small batch size for testing
node_size = cfg.dataset.node_sz
input_dim = int(node_size * (node_size - 1) / 2)
random_corr_vector = torch.randn(batch_size, input_dim)

In [8]:
random_corr_vector.shape

torch.Size([4, 19900])

In [9]:
mlp_model = MLP(cfg)
mlp_model.eval()  # Set the model to evaluation mode

MLP(
  (network): Sequential(
    (0): Linear(in_features=19900, out_features=25, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=25, out_features=12, bias=True)
    (4): LeakyReLU(negative_slope=0.01)
    (5): Dropout(p=0.1, inplace=False)
  )
  (fc): Linear(in_features=12, out_features=2, bias=True)
)

In [10]:
print("Performing forward pass...")
with torch.no_grad(): # We don't need to calculate gradients for this test
    output = mlp_model(random_corr_vector)
print(f"Received output with shape: {output.shape}")
print(f"First output: {output[0]}")

Performing forward pass...
Received output with shape: torch.Size([4, 2])
First output: tensor([-0.2939,  0.1327])
